**Prj02.** (L04) This is an application of BSM evaluation to Geometric asian option price

Geometric asian call option with maturity $T$ and strike $K$ has its pay off as
$$C(T) = (A(T) - K)^+,$$
where $A(T)$ is geometric average of the stock price at times
$0 \le t_1 < t_2, \ldots, < t_n = T$, i.e.
$$A(T) = (S(t_1)S(t_2) \ldots S(t_n))^{1/n}.$$

The call price can be thus written by
$$C_0 = \mathbb E [e^{-rT} (A(T) - K)^+].$$

**To do**
Use "BSM_option_valuation" module to find the BSM asian option value with the following parameters



In [1]:
%reset -f
import numpy as np
#An example is given here
S0 = 100.0 #initial stock price
K = 110.0 #strike
r=0.0475 #interest rate
sigma = 0.20 #vol
T = 1. #maturity
Otype='C' #Call type
n = 4 #number of periods
t = np.linspace(0., T, n+1)[1:] #times to be used for geometric averaging stock price

**Hint** 

Under the above BS model, one can show that the distribution of $A(T)$ is again a lognormal under EMM in the form of
$$A_T = S_0 \exp \{ (\hat r - \frac 1 2 \hat \sigma^2)T + \hat \sigma \sqrt T \hat Z\}.$$
In the above, $\hat Z$ is a standard normal random variable, $\hat \sigma$ is
$$\hat \sigma = \frac{\sigma}{n} \sqrt{\frac{(n+1)(2n+1)}{6}}$$
and $\hat r$ is
$$\hat r = \frac 1 2 \hat \sigma^2 + \frac{n+1}{2n}(r - \frac 1 2 \sigma^2).$$
Then, by fundamental asset pricing theory, we have GAC price by
$$\Pi_0^c = e^{(\hat r - r)T} \mathbb E [ e^{-\hat r T} (A_T - K)^+].$$


---

* ***BSM_option_valuation***

In [2]:
import numpy as np
import scipy.stats as ss
import time 
import math

In [3]:
def d1f(St, K, t, T, r, sigma):
    ''' Black-Scholes-Merton d1 function.
        Parameters see e.g. BSM_call_value function. '''
    d1 = (math.log(St / K) + (r + 0.5 * sigma ** 2)
          * (T - t)) / (sigma * math.sqrt(T - t))
    return d1

In [4]:
def BSM_call_value(St, K, t, T, r, sigma):
    ''' Calculates Black-Scholes-Merton European call option value.

    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    sigma : float
        volatility

    Returns
    =======
    call_value : float
        European call present value at t
    '''
    d1 = d1f(St, K, t, T, r, sigma)
    d2 = d1 - sigma * math.sqrt(T - t)
    call_value = St * ss.norm.cdf(d1) - math.exp(-r * (T - t)) * K * ss.norm.cdf(d2)
    return call_value

def BSM_put_value(St, K, t, T, r, sigma):
    ''' Calculates Black-Scholes-Merton European put option value.

    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    sigma : float
        volatility

    Returns
    =======
    put_value : float
        European put present value at t
    '''
    put_value = BSM_call_value(St, K, t, T, r, sigma) \
        - St + math.exp(-r * (T - t)) * K
    return put_value

* ***Asian parameters & GAC price***
$$\hat \sigma = \frac{\sigma}{n} \sqrt{\frac{(n+1)(2n+1)}{6}}$$
$$\hat r = \frac 1 2 \hat \sigma^2 + \frac{n+1}{2n}(r - \frac 1 2 \sigma^2).$$
$$\Pi_0^c = e^{(\hat r - r)T} \mathbb E [ e^{-\hat r T} (A_T - K)^+].$$

In [15]:
sigma_hat = sigma / n * np.sqrt ((n + 1) * (2 * n + 1) / 6)
r_hat = 0.5 * sigma_hat ** 2 + (n + 1) / (2 * n) * (r - 0.5 * sigma ** 2)
print("sigma_hat is %s \n " %(sigma_hat),"r_hat is %s"%(r_hat))

sigma_hat is 0.13693063937629155 
  r_hat is 0.026562500000000003


# BSM asian call option value

In [18]:
GAC_price = np.exp((r_hat - r) * (T - 0)) * BSM_call_value(S0, K, 0, T, r_hat, sigma_hat)
print("the price of GAC at t0 is", GAC_price)

the price of GAC at t0 is 2.7329867250697175
